# Week 2 Discussion: Estimators

In [0]:
# Ensures consistent results from our simulations
set.seed(125)

# Load in tidyverse packages
library(tidyverse)

# Make our `ggplot` plots use a same black and white theme
theme_set(theme_bw())

## Setup

You're given a huge jar of Skittles and M&M's, mixed together (yuck!).

![skittles](img/mm_skittles.jpg)

You randomly pick 10 pieces of candy from the jar, and end up with 4 Skittles 
and 6 M&M's.

**What proportion of the jar do you think are M&M's?**

A natural, intuitive, **estimate** for the proportion of M&M's in the jar would be

$$\frac{6}{4 + 6} = 0.6$$

But what can we say about this number? 
How certain are we that if we count _all_ the candy in the jar, the proportion
of M&M's will be close to 60%?

To answer these questions, we have to think more carefully about how that
**0.6** came about, and what it means.

## Estimator

Let's say there are $C$ pieces of candy in the jar and $M$ pieces are
M&M's (i.e., there are $C-M$ Skittles in the jar). We'll call the actual
proportion of M&M's in the jar $p$, which is exactly $M/C$. 

Then, taking out one candy from the jar and inspecting whether it's an M&M or
Skittle is equivalent to a Bernoulli trial with probability $p$ of "success".
We can also do this multiple times, and each time would be an independent 
Bernoulli trial, as long as we put the candy back in the jar every time.

For each piece of candy that we inspect, let's write a 1 if it's an M&M and a 
0 if it's a Skittle, and we'll call the $i^\textrm{th}$ number we write $X_i$.
Then we can say that the **0.6** above comes from using the following **estimator**:

$$\hat{p} = \frac{1}{N}\sum_{i = 1}^{N}X_i$$

with our observed data ($X_1, X_2, \cdots X_{10}$), where $N$ is the number of
pieces of candy we get to count.


Now, to answer questions regarding our (un)certainty about that **0.6**
number, we need to figure out the **distribution** of our estimator,
$\hat{p}$.

## Interlude: Distributions in `R`

As you've seen in introductory probability, it's common to 
represent natural phenomena in terms of known probability distributions.
For example, in a particular population, the height of individuals may
be normally distribution with a mean $\mu$ of 5.5 feet and a standard
deviation $\sigma$ of 0.5 feet. 

You can work with known distributions (e.g., Normal, Poisson, and
Binomial) in `R` using the following notation:
`<d,p,q,r><distribution_name>`

For example, `dnorm`, `pbinom`, `qpoisson`, and `rbeta` are valid `R` commands. 

What do the four letters stand for?

* d: density (i.e., the height of the distribution at a particular `x` value)
* p: probability (i.e., the area under the distribution from $-\infty$ to `x`)
* q: quantile (i.e., the `x` value corresponding to a particular `p` [`p` is the area under the distribution from $-\infty$ to `x`])
* r: random (i.e., generate `n` random draws of `x` values from the distribution)

### Examples

Generate 10 random draws from the standard normal distribution.

In [0]:
# Equivalently, rnorm(10)
rnorm(n = 10, mean = 0, sd = 1)

What is the area under the standard normal distribution
from 1 standard deviation below the mean to 2 standard deviations
above the mean?

In [0]:
# Equivalently, pnorm(2) - pnorm(-1)
pnorm(q = 2, mean = 0, sd = 1) - pnorm(q = -1, mean = 0, sd = 1)

### Exercises

> **Simulate a carnival game where I have 10 guesses to win a prize, and, 
for each guess, the probability of winning a prize is 20%. 
You can win multiple prizes. Run your code several times. What was
the minimum number of times you won? The maximum?**

In [0]:
# Your code here! 



> **What value corresponds to the 97.5th percentile of the standard normal distribution?**

In [0]:
# Your code here! 



## Distribution of an estimator 

### A simulation

Before diving into distributions with more notation, symbols, and all that, 
let's think more carefully about what "**the distribution of** $\hat{p}$" 
actually means. For this, we'll pretend we know exactly how many M&M's and 
Skittles are in the jar, and see what happens when we compute values of $\hat{p}$.

First, let's create that jar:

In [0]:
M <- 1000   # Let's say this is the number of  M&M's
S <- 2000   # and this is the number of Skittles.
C <- M + S  # Then this is the total 
print(C)

and the actual proportion of M&M's ($p$) is simply:

In [0]:
p <- M/C
print(p)

Then, randomly inspecting 10 pieces of candy from the jar is equivalent to 10 Bernoulli trials with probability $p$ of success, i.e.,

In [0]:
N <- 10  # This is how many we get to count
counted_candies <- rbinom(N, 1, p)
print(counted_candies)

Remember, drawing an M&M is considered a "success" in this setting, so we can compute our estimator $\hat{p}$ with

In [0]:
p_hat <- sum(counted_candies)/N
print(p_hat)

The mean of a vector of 0s and 1s is the proportion of elements in the vector that are 1, so we can also do this:

In [0]:
p_hat <- mean(counted_candies)
print(p_hat)

But that $\hat{p}$ is just a number! How do we find its distribution? What does a distribution for $\hat{p}$ even mean?!

### Sampling distributions

The distribution of an estimator (also commonly known as the sampling
distribution) refers to how the values of $\hat{p}$ would look like
**if we were to repeat the counting for a whole bunch of samples (of the same size)**.

In other words, if there were multiple parallel universes,
what would the proportion of M&M's in each
of those samples of 10 candies look like across all the universes?

While it might be a little difficult to create a bunch of parallel universes in
practice, this can easily be done in `R`:

In [0]:
B <- 100  # Number of parallel universes to create
multiverse_p_hats <- replicate(B, mean(rbinom(N, 1, p)))
print(multiverse_p_hats)

#### Exercise
> **Now that we have `B` values of $\hat{p}$ from `B` parallel universes,
> make a histogram of the $\hat{p}$ values. You may want to use
> the `tibble` function to create a dataframe, and you may want to 
> adjust the default `binwidth` parameter of 
> `geom_histogram`.**

In [0]:
# Your code here!



#### Properties of sampling distributions

While we know the true value of $p$ (because we created it out of thin air!), we see that the value of $\hat{p}$ varies across all `B` universes. 

However, the _average_ of $\hat{p}$ (the estimated expected value of $\hat{p}$) across all those universes is actually not that far from $p$.

In [0]:
p # actual value of p
mean(multiverse_p_hats)  # Estimated expected value of p_hat
mean(multiverse_p_hats) - p  # => quite small!

This difference between the actual value of interest ($p$) and the expected 
value of an estimator (average across multiple universes) is what's known as
the **bias of an estimator**:

$$\mathbb{E}_p(\hat{p}) - p$$

We can also estimate the standard deviation of our estimator $\hat{p}$ across
multiple universes (often called the estimated **standard error**):

In [0]:
sd(multiverse_p_hats)

### The math


We can also compute the properties of our estimator $\hat{p}$ analytically,
without simulation (which is the kind of analysis we expect for Problem 1a and 1b of 
[homework 2](https://5harad.com/mse125/#hw2)).

First, we have to start with basic probability.
Remember, what we observe is $X_i$, and each of these $X_i$ all follow
(independently and identically) a Bernoulli distribution with probability of
success $p$. Then, from the [properties of a Bernoulli
distribution](https://en.wikipedia.org/wiki/Bernoulli_distribution) we know that

\begin{align*}
\mathbb{E}_p(X_i) & = p \\
\mathrm{Var}_p(X_i) & = p(1-p) \\
\end{align*}

where the subscript $_p$ in 
$\mathbb{E}_p$ and $\mathrm{Var}_p$ simply means that the actual value $p$ is fixed (i.e., it's _not_ random).

**Linearity of expectation** implies the following:

$$\mathbb{E} \left( \sum_{i=1}^n X_i \right) = \sum_{i=1}^n \mathbb{E}(X_i)$$
Recall our estimator:

$$\hat{p} = \frac{1}{N}\sum_{i = 1}^{N}X_i$$

#### Exercise 

> **Using the facts above, derive an expression for $\mathbb{E}_p(\hat{p})$.**

Click here to write your answer in $\LaTeX$, or write your answer on a separate sheet of paper.


<!-- \begin{align*}
\mathbb{E}_p(\hat{p}) & = \mathbb{E}_p\left(\frac{1}{N}\sum_{i = 1}^{N}X_i\right) \\
  & = \frac{1}{N}\mathbb{E}_p\left(\sum_{i = 1}^{N}X_i\right) \\
  & = \frac{1}{N}\sum_{i = 1}^{N}\mathbb{E}_p(X_i) \\
  & = \frac{1}{N}\sum_{i = 1}^{N}p \\
  & = \frac{1}{N}Np \\
  & = p
\end{align*} -->

#### Bias

For the bias, we want to figure out the value of

$$\mathbb{E}_p(\hat{p}) - p$$

$\mathbb{E}_p(\hat{p}) - p = 0$, so we'd call $\hat{p}$ **unbiased**. 

This is confirmed in our simulation above, where we find that the average of our $\hat{p}$ estimates across `B` multiverses is very close to the true value of $p$!

#### Standard error

Again, what's known as the **standard error** is simply the **standard deviation of the sampling distribution**, where the _sampling distribution_ is the distribution of values that our estimator $\hat{p}$ would take across multiple parallel universes.

In other words, we want to know

$$\sqrt{\mathrm{Var}_p(\hat{p})}$$

In case it's not clear yet, $\hat{p}$ is a **random variable** that can take multiple values, depending on the exact samples ($X_i$) we happened to get, and $p$ is a real, fixed value (the proportion of M&M's in the jar) that won't change no matter how many candies we sample (unless we eat them).

### Exercise

> **Using the facts stated at the beginning of this section,
> derive an expression for $\mathrm{Var}_p(\hat{p})$.**

Click here to write your answer in $\LaTeX$, or write your answer on a separate sheet of paper.


<!-- \begin{align*}
\mathrm{Var}_p(\hat{p})
  & = \mathrm{Var}_p\left(\frac{1}{N}\sum_{i = 1}^{N}X_i\right) \\
  & = \frac{1}{N^2}\sum_{i = 1}^{N}\mathrm{Var}_p(X_i) \\
  & = \frac{1}{N^2}Np(1-p) \\
  & = \frac{p(1-p)}{N}
\end{align*}

and the standard error is:

$$\sqrt{\mathrm{Var}_p(\hat{p})} = \sqrt{\frac{p(1-p)}{N}}$$ -->

We can confirm our result with the standard deviation of our estimates 
across multiverses, i.e.,

In [0]:
# This is what the theory predicts we'll see
sqrt(p*(1-p)/N) 
# This is what we get from parallel universes
sd(multiverse_p_hats)

But note that in reality, we couldn't really compute the theoretical standard error, since it requires knowing the value of $p$, which in this case, we know _only because we created it out of thin air_. 

So, in reality, we'd _estimate_ the standard error, by using our estimated value of $\hat{p}$, in place of the true value $p$. For example, in this case we'd compute

$$\hat{\mathrm{se}} =  \sqrt{\frac{\hat{p}(1-\hat{p})}{N}}$$

### Exercise

> **Calculate $\hat{\mathrm{se}}$ for the very first example of 6 M&M's in a sample of 10 candies.**

In [0]:
# Your code here!



## Confidence intervals 

Now we have some understanding of the distribution of our estimator.
Given this information, what more can we say about our estimate, $\hat{p} = 0.6$?
Well, since we now know that there is some uncertainty associated with it, and we have some measure of that uncertainty (the estimated standard error), we can express our uncertainty in terms of a range of values, instead of trying
to pin-point an exact value that we know won't be quite right.
One way to do this is to construct a **confidence interval**.

One simple yet common approximation for a confidence interval is to assume that the sampling distribution is close to a normal distribution, and start from there.

As discussed in lecture, a $1-\alpha$ confidence interval for our estimator $\hat{p}$ can be constructed as:

\begin{align*}
\mathrm{CI} 
  & = \left(\hat{p} - z_{1 - \alpha/2}\mathrm{se}(\hat{p}), 
      \hat{p} + z_{1 - \alpha/2}\mathrm{se}(\hat{p})\right) \\
  & \approx \left(\hat{p} - z_{1 - \alpha/2}\hat{\mathrm{se}}(\hat{p}), 
      \hat{p} + z_{1 - \alpha/2}\hat{\mathrm{se}}(\hat{p})\right) \\
  & = \left(\hat{p} - z_{1 - \alpha/2}\sqrt{\hat{p}(1-\hat{p})/N},  
             \hat{p} + z_{1 - \alpha/2}\sqrt{\hat{p}(1-\hat{p})/N}\right)
\end{align*}

where $z_{1 - \alpha/2}$ is the value at which the CDF of a standard normal distribution is equal $1 - \alpha/2$.

![distribution](img/alpha-normal-distr.png)

### Exercise

> **Construct a 95% confidence interval for our initial estimate of $\hat{p} = 0.6$. Note that we drew 10 candies to calculate this estimate.**

In [0]:
# Your code here!



### Validation via simulation

But what does this confidence interval mean?
Well, to continue our multiverse analogy, a 95% confidence interval would mean that if we were to follow the exact same procedure in all of our parallel universes, **the true value of $p$ would be contained within the computed interval in about 95% of the universes.**

This can be a little confusing, so let's try to clarify with some more simulation.
Remember, in our parallel universe example, we actually _know_ the true value of  $p$, so we can check whether any interval created in any of the universes successfully contains that value or not. 
For convenience, let's start by creating a function that carries out our process of computing confidence intervals from a set of observations:

In [0]:
# X: the vector of observed M&Ms (1s) and Skittles (0s)
compute_ci <- function(X, alpha = 1 - .95) {
  N <- length(X)
  p_hat <- mean(X)
  se <- sqrt(p_hat * (1 - p_hat) / N)
  
  c(p_hat - qnorm(1 - alpha/2)*se, 
    p_hat + qnorm(1 - alpha/2)*se)
}

Let's also write a function that, given a confidence interval and a true value, returns `TRUE` if the true value is contained in the interval, and `FALSE` otherwise:

In [0]:
in_interval <- function(ci, p) {
  # This works, but can be improved
  # (e.g., we assume ci is sorted)
  if (ci[1] <= p & ci[2] >= p) {
    return(TRUE)
  }
  return(FALSE)
}

Now, we are ready to replicate parallel universes, 
generate confidence intervals, and see whether they contain the true value.

### Exercise

> **Code up a simulation that generates confidence intervals for 100 universes, and calculates the proportion of universes for which the confidence interval contains the true value of $p$.**

In [0]:
# Your code here!



### Exercise

> **Ideally, the above simulation result should be close to `0.95`---because we constructed 95% confidence intervals; but it probably isn't. Why do you think this happened? What could you change above in that last block to make the simulation result closer to `0.95`?**

> **(Hint: Think about the histogram from our previous simulation. Based
on the shape of the histogram, are any assumptions violated?)**

In [0]:
# Your answer/code here!

